In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# --- 1. AUTO-FIND THE PATH ---
print("🕵️‍♂️ Searching for 'training_videos'...")

found_train_path = ""
# Walk through the entire input directory to find the folder
for root, dirs, files in os.walk('/kaggle/input'):
    if "training_videos" in dirs:
        found_train_path = os.path.join(root, 'training_videos')
        print(f"✅ FOUND IT! The correct path is:\n{found_train_path}")
        break

if found_train_path == "":
    print("❌ ERROR: Could not find the folder. Check if the dataset is added!")
else:
    # --- 2. LIST FOLDERS ---
    video_folders = sorted(os.listdir(found_train_path))
    print(f"\n📂 Found {len(video_folders)} video folders (e.g., {video_folders[:3]}...)")

    # --- 3. SHOW AN IMAGE ---
    # Go inside the first video folder
    first_video_path = os.path.join(found_train_path, video_folders[0])
    frame_files = sorted(os.listdir(first_video_path))
    
    # Pick the first image
    img_path = os.path.join(first_video_path, frame_files[0])
    
    # Display
    img = mpimg.imread(img_path)
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.title(f"Success! Viewing: {video_folders[0]}/{frame_files[0]}")
    plt.axis('off')
    plt.show()
    
    print(f"Image Shape: {img.shape}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import os

# --- CONFIGURATION ---
TRAIN_DIR = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/training_videos'
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

# 1. Load raw images
raw_ds = image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode=None, 
    shuffle=True
)

# 2. Preprocessing: Normalization and Autoencoder Pair (Input, Target)
def process_data(images):
    # Normalize to [0, 1]
    norm_imgs = tf.cast(images, tf.float32) / 255.0
    # For an autoencoder, the input and the target are the same
    return norm_imgs, norm_imgs

train_generator = (
    raw_ds
    .map(process_data, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

print("✅ Preprocessing complete. Ready for high-capacity training!")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications

def build_semantic_backbone():
    # ResNet-101 (R101) for semantic feature extraction
    base_model = applications.ResNet101(
        input_shape=(128, 128, 3), 
        include_top=False, 
        weights='imagenet'
    )
    base_model.trainable = False 

    # GAP layer converts convolutional maps into high-level features
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D() 
    ])
    return model

feature_extractor = build_semantic_backbone()
print("🚀 Optimized Backbone Ready (ResNet-101)")

In [ ]:
from tensorflow.keras import layers, models

def build_optimized_dae(input_dim=2048):
    inputs = layers.Input(shape=(input_dim,))

    # -------------------------------------------------
    # Add noise to UNCORRUPTED data 
    # -------------------------------------------------
    x = layers.GaussianNoise(0.1)(inputs)

    # -------------------------------------------------
    # Wide INPUT layer (NEW)
    # -------------------------------------------------
    x = layers.Dense(1536)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('sigmoid')(x)

    # -------------------------------------------------
    # Encoder (UNCHANGED)
    # -------------------------------------------------
    x = layers.Dense(50)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('sigmoid')(x)

    # -------------------------------------------------
    # Code layer (UNCHANGED)
    # -------------------------------------------------
    code = layers.Dense(30)(x)
    code = layers.BatchNormalization()(code)
    code = layers.Activation('sigmoid')(code)

    # -------------------------------------------------
    # Decoder (UNCHANGED)
    # -------------------------------------------------
    x = layers.Dense(50)(code)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('sigmoid')(x)

    # -------------------------------------------------
    # Wide OUTPUT layer (NEW)
    # -------------------------------------------------
    x = layers.Dense(1536)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Activation('sigmoid')(code) no sigmoid in output

    # -------------------------------------------------
    # Final reconstruction (UNCHANGED)
    # -------------------------------------------------
    outputs = layers.Dense(input_dim, activation='linear')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

dae_model = build_optimized_dae()
print("✅ Optimized DAE with wide input/output built")
dae_model.summary()


In [ ]:
import numpy as np

# 1. Faster Training Data Loader
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, image_size=(128, 128), batch_size=128, label_mode=None, shuffle=True
)

# 2. Optimized Extraction Phase (Maximum P100 throughput)
print("--- Phase 1: High-Speed Semantic Extraction ---")
all_train_features = []
for batch in train_ds:
    batch_norm = tf.cast(batch, tf.float32) / 255.0
    # Batch predict is faster than single predict 
    feats = feature_extractor.predict(batch_norm, verbose=0)
    all_train_features.append(feats)

train_vectors = np.vstack(all_train_features)

# 3. DAE Training (Converges in seconds with semantic vectors) 
print(f"--- Phase 2: Training on {train_vectors.shape[1]}-dim semantic space ---")
dae_model.fit(
    train_vectors, train_vectors, 
    epochs=150, batch_size=120, verbose=1 # Batch size 120 from paper 
)

dae_model.save('paper_optimized_model.keras')

In [ ]:
import re, os
import pandas as pd
from scipy.stats import rankdata
# import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d

TEST_PATH='/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos'
# 1. Config Test Loader
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH, image_size=(128, 128), batch_size=64, label_mode=None, shuffle=False
)
test_filenames = test_ds.file_paths

# 2. Batched Reconstruction Error Calculation 
all_errors = []
print(f"--- Processing {len(test_filenames)} test frames in batches ---")

for batch in test_ds:
    batch_norm = tf.cast(batch, tf.float32) / 255.0
    # Process 64 images at once
    feats = feature_extractor.predict(batch_norm, verbose=0)
    recons = dae_model.predict(feats, verbose=0)
    
    # Calculate MSE reconstruction error for the entire batch 
    batch_mse = np.mean((feats - recons)**2, axis=1)
    all_errors.extend(batch_mse)

# 3. Paper-Based ID Formatting (1_939)
def get_clean_id(p):
    parts = p.split(os.sep)
    f_num = int(re.findall(r'\d+', parts[-2])[-1])
    i_num = int(re.findall(r'\d+', parts[-1])[-1])
    return f"{f_num}_{i_num}"

# 4. Save Final Submission
submission_df = pd.DataFrame({
    'Id': [get_clean_id(p) for p in test_filenames],
    'Predicted': rankdata(all_errors) / len(all_errors)
})



# 1. Load your baseline
df = submission_df
df[['Video', 'Frame']] = df['Id'].str.split('_', expand=True).astype(int)
df = df.sort_values(['Video', 'Frame'])

# 2. MSE -> PSNR Conversion
# Peak Signal-to-Noise Ratio focuses on the reconstruction quality
epsilon = 1e-10
df['PSNR'] = 10 * np.log10(1.0 / (df['Predicted'] + epsilon))

# 3. Normalization Strategy
STRATEGY = 'GLOBAL' 

if STRATEGY == 'PER_VIDEO':
    def norm_func(x):
        return 1 - ((x - x.min()) / (x.max() - x.min() + epsilon))
    df['Anomaly_Score'] = df.groupby('Video')['PSNR'].transform(norm_func)
else:
    # Global normalization preserves the 'intensity' difference between videos
    p_min, p_max = df['PSNR'].min(), df['PSNR'].max()
    df['Anomaly_Score'] = 1 - ((df['PSNR'] - p_min) / (p_max - p_min + epsilon))

# 4. Gaussian Smoothing (More precise than Moving Average)
df['Smoothed'] = df.groupby('Video')['Anomaly_Score'].transform(
    lambda x: gaussian_filter1d(x, sigma=3)
)

# 5. Save the two best candidates
# df[['Id', 'Anomaly_Score']].rename(columns={'Anomaly_Score': 'Predicted'}).to_csv('sub_psnr_only.csv', index=False)
df[['Id', 'Smoothed']].rename(columns={'Smoothed': 'Predicted'}).to_csv('sub_psnr_gaussian.csv', index=False)

print('sub_psnr_gaussian.csv')



# submission_df.to_csv('submission_paper_semantic_optimized.csv', index=False)
# print("🚀 Optimized Submission Saved!")

In [ ]:
import re, os
import pandas as pd
from scipy.stats import rankdata
TEST_PATH='/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos'
# 1. Config Test Loader
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH, image_size=(128, 128), batch_size=64, label_mode=None, shuffle=False
)
test_filenames = test_ds.file_paths

# 2. Batched Reconstruction Error Calculation [cite: 268, 271]
all_errors = []
print(f"--- Processing {len(test_filenames)} test frames in batches ---")

for batch in test_ds:
    batch_norm = tf.cast(batch, tf.float32) / 255.0
    # Process 64 images at once
    feats = feature_extractor.predict(batch_norm, verbose=0)
    recons = dae_model.predict(feats, verbose=0)
    
    # Calculate MSE reconstruction error for the entire batch [cite: 268, 271]
    batch_mse = np.mean((feats - recons)**2, axis=1)
    all_errors.extend(batch_mse)

# 3. Paper-Based ID Formatting (1_939)
def get_clean_id(p):
    parts = p.split(os.sep)
    f_num = int(re.findall(r'\d+', parts[-2])[-1])
    i_num = int(re.findall(r'\d+', parts[-1])[-1])
    return f"{f_num}_{i_num}"

# 4. Save Final 0.85 Target Submission
submission_df = pd.DataFrame({
    'Id': [get_clean_id(p) for p in test_filenames],
    'Predicted': rankdata(all_errors) / len(all_errors)
})

submission_df.to_csv('submission_paper_semantic_optimized.csv', index=False)
print("🚀 Optimized Submission Saved!")

In [ ]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d

# 1. Load your best baseline
df = pd.read_csv('/kaggle/working/submission_paper_semantic_optimized.csv')
df[['Video', 'Frame']] = df['Id'].str.split('_', expand=True).astype(int)
df = df.sort_values(['Video', 'Frame'])

# 2. MSE -> PSNR Conversion
# Peak Signal-to-Noise Ratio focuses on the reconstruction quality
epsilon = 1e-10
df['PSNR'] = 10 * np.log10(1.0 / (df['Predicted'] + epsilon))

# 3. Choose your Normalization Strategy
# If Per-Video lowered your score before, try 'GLOBAL'
STRATEGY = 'GLOBAL' 

if STRATEGY == 'PER_VIDEO':
    def norm_func(x):
        return 1 - ((x - x.min()) / (x.max() - x.min() + epsilon))
    df['Anomaly_Score'] = df.groupby('Video')['PSNR'].transform(norm_func)
else:
    # Global normalization preserves the 'intensity' difference between videos
    p_min, p_max = df['PSNR'].min(), df['PSNR'].max()
    df['Anomaly_Score'] = 1 - ((df['PSNR'] - p_min) / (p_max - p_min + epsilon))

# 4. Gaussian Smoothing (More precise than Moving Average)
# sigma=1.0 or 1.5 is the 'sweet spot' for 24-30fps video
df['Smoothed'] = df.groupby('Video')['Anomaly_Score'].transform(
    lambda x: gaussian_filter1d(x, sigma=1.2)
)

# 5. Save the two best candidates
df[['Id', 'Anomaly_Score']].rename(columns={'Anomaly_Score': 'Predicted'}).to_csv('sub_psnr_only.csv', index=False)
df[['Id', 'Smoothed']].rename(columns={'Smoothed': 'Predicted'}).to_csv('sub_psnr_gaussian.csv', index=False)

print("Generated 'sub_psnr_only.csv' and 'sub_psnr_gaussian.csv'")